In [36]:
from airplane import Airplane
from receiver import Receiver

airplane1 = Airplane('782034',[120.128234,30.2141348],[115.86143245,28.750012],180,1564503340)
receiver1 = Receiver([114.122588,36.548925])
time_track1 =receiver1.fin_time_track(airplane1)
track1 = airplane1.gettrack()

# 经纬度数据归一化

In [37]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(-1,1))


# train_set_array = np.array(train_set)
# train_norm = sc.fit_transform(train_set_array.reshape(-1,1))


lng = []
lat = []

for position in track1:
    lng.append(position[0])
    lat.append(position[1])

lng_array = np.array(lng)
lng_norm = sc.fit_transform(lng_array.reshape(-1,1))
print(lng_norm)

lat_array = np.array(lat)
lat_norm = sc.fit_transform(lat_array.reshape(-1,1))

track_norm = [] #归一化轨迹列表
for i in range(len(lng_norm)):
    track_norm.append([lng_norm[i][0],lat_norm[i][0]])


    
    

[[ 1.        ]
 [ 0.9995944 ]
 [ 0.99918881]
 ...
 [-0.99918881]
 [-0.9995944 ]
 [-1.        ]]


In [38]:
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt,pi, atan

# using_data = time_track1
# test_data = []#前后两两时间差

# for i in range(len(using_data)-1):
#     test_data.append(using_data[i+1]-using_data[i]-airplane1.geodistance(track1[i+1],receiver1.location)*10/3)


# plt.plot(using_data[:-1], test_data)
# plt.show()

# receiver1.plt_location(airplane1)

In [39]:
# len(using_data)
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

track_norm=track_norm[:4000]
test_size = 25
train_set = track_norm[: -test_size]
test_set = track_norm[-test_size:]

print(train_set)
print(test_set)

train_norm = torch.Tensor(track_norm).cuda()

[[1.0, 1.0], [0.9995944027580634, 0.9995944027580634], [0.9991888055161269, 0.9991888055161269], [0.9987832082741903, 0.9987832082741903], [0.9983776110322466, 0.9983776110322466], [0.9979720137903101, 0.9979720137903101], [0.9975664165483735, 0.9975664165483735], [0.997160819306437, 0.9971608193064299], [0.9967552220644933, 0.9967552220644933], [0.9963496248225567, 0.9963496248225567], [0.9959440275806202, 0.9959440275806202], [0.9955384303386765, 0.9955384303386765], [0.9951328330967328, 0.9951328330967399], [0.9947272358547963, 0.9947272358548034], [0.9943216386128597, 0.9943216386128597], [0.993916041370916, 0.9939160413709232], [0.9935104441289795, 0.9935104441289795], [0.9931048468870429, 0.9931048468870429], [0.9926992496451064, 0.9926992496451064], [0.9922936524031627, 0.9922936524031627], [0.9918880551612261, 0.9918880551612332], [0.9914824579192896, 0.9914824579192896], [0.991076860677353, 0.991076860677353], [0.9906712634354093, 0.9906712634354093], [0.9902656661934728, 0.99

In [40]:
def input_data(seq,ws):
    out = []
    L = len(seq)

    for i in range(L-ws):
        window = seq[i : i+ws]
        label = seq[i+ws : i+ws+1]
        out.append((window,label))

    return out



window_size = 25
train_data = input_data(train_norm, window_size)
# print(train_data)
print(len(train_data))
print(train_data[0])

3975
(tensor([[1.0000, 1.0000],
        [0.9996, 0.9996],
        [0.9992, 0.9992],
        [0.9988, 0.9988],
        [0.9984, 0.9984],
        [0.9980, 0.9980],
        [0.9976, 0.9976],
        [0.9972, 0.9972],
        [0.9968, 0.9968],
        [0.9963, 0.9963],
        [0.9959, 0.9959],
        [0.9955, 0.9955],
        [0.9951, 0.9951],
        [0.9947, 0.9947],
        [0.9943, 0.9943],
        [0.9939, 0.9939],
        [0.9935, 0.9935],
        [0.9931, 0.9931],
        [0.9927, 0.9927],
        [0.9923, 0.9923],
        [0.9919, 0.9919],
        [0.9915, 0.9915],
        [0.9911, 0.9911],
        [0.9907, 0.9907],
        [0.9903, 0.9903]], device='cuda:0'), tensor([[0.9899, 0.9899]], device='cuda:0'))


In [41]:
class LSTM(nn.Module):
    def __init__(self, input_size=2, hidden_size=100,out_size=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size,out_size)
        self.hidden = ( torch.zeros(1, 1, hidden_size), torch.zeros(1, 1, hidden_size))

    def forward(self,seq):
        lstm_out, self.hidden = self.lstm(seq.view(len(seq), 1, -1), self.hidden)
        pred= self.linear(lstm_out.view(len(seq), -1))

        return pred[-1]


torch.manual_seed(42)
model = LSTM().cuda()
# train_data.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.005)


print(model)


for p in model.parameters():
    print(p.numel())

LSTM(
  (lstm): LSTM(2, 100)
  (linear): Linear(in_features=100, out_features=2, bias=True)
)
800
40000
400
400
200
2


In [42]:
epochs = 50

for i in range(epochs):
    for seq, y_train in train_data:
        optimizer.zero_grad()
        model.hidden = (torch.zeros(1,1, model.hidden_size).cuda(), torch.zeros(1,1, model.hidden_size).cuda())
        y_pred = model(seq)

        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
    
    
    
    print( f" Epoch {i} Loss:  {loss.item()}")


/home/ubuntu/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


 Epoch 0 Loss:  0.0012189073022454977
 Epoch 1 Loss:  0.0001372696424368769
 Epoch 2 Loss:  1.7113230569520965e-05
 Epoch 3 Loss:  9.612120265956037e-06
 Epoch 4 Loss:  8.942407475842629e-06
 Epoch 5 Loss:  8.676219295011833e-06
 Epoch 6 Loss:  8.444396371487528e-06


KeyboardInterrupt: 

In [43]:
future = 25
preds = train_norm[-window_size:].tolist()

print(preds)
model.eval()

for i in range(future):
    seq = torch.Tensor(preds[-window_size:]).cuda()
    with torch.no_grad():
        model.hidden = (torch.zeros(1,1,model.hidden_size).cuda(),torch.zeros(1,1,model.hidden_size).cuda())
        preds.append(model(seq).tolist())

# print(preds)

[[-0.6122490167617798, -0.6122490167617798], [-0.6126546263694763, -0.6126546263694763], [-0.6130602359771729, -0.6130602359771729], [-0.6134658455848694, -0.6134658455848694], [-0.6138714551925659, -0.6138714551925659], [-0.6142770051956177, -0.6142770051956177], [-0.6146826148033142, -0.6146826148033142], [-0.6150882244110107, -0.6150882244110107], [-0.6154938340187073, -0.6154938340187073], [-0.615899384021759, -0.615899384021759], [-0.6163049936294556, -0.6163049936294556], [-0.6167106032371521, -0.6167106032371521], [-0.6171162128448486, -0.6171162128448486], [-0.6175218224525452, -0.6175218224525452], [-0.6179273724555969, -0.6179273724555969], [-0.6183329820632935, -0.6183329820632935], [-0.61873859167099, -0.61873859167099], [-0.6191442012786865, -0.6191442012786865], [-0.6195498108863831, -0.6195498108863831], [-0.6199553608894348, -0.6199553608894348], [-0.6203609704971313, -0.6203609704971313], [-0.6207665801048279, -0.6207665801048279], [-0.6211721897125244, -0.621172189712

In [46]:
print(torch.Tensor(preds))
print(torch.Tensor(test_set))

tensor([[-0.6122, -0.6122],
        [-0.6127, -0.6127],
        [-0.6131, -0.6131],
        [-0.6135, -0.6135],
        [-0.6139, -0.6139],
        [-0.6143, -0.6143],
        [-0.6147, -0.6147],
        [-0.6151, -0.6151],
        [-0.6155, -0.6155],
        [-0.6159, -0.6159],
        [-0.6163, -0.6163],
        [-0.6167, -0.6167],
        [-0.6171, -0.6171],
        [-0.6175, -0.6175],
        [-0.6179, -0.6179],
        [-0.6183, -0.6183],
        [-0.6187, -0.6187],
        [-0.6191, -0.6191],
        [-0.6195, -0.6195],
        [-0.6200, -0.6200],
        [-0.6204, -0.6204],
        [-0.6208, -0.6208],
        [-0.6212, -0.6212],
        [-0.6216, -0.6216],
        [-0.6220, -0.6220],
        [-0.6232, -0.6240],
        [-0.6239, -0.6247],
        [-0.6245, -0.6253],
        [-0.6251, -0.6260],
        [-0.6257, -0.6265],
        [-0.6263, -0.6271],
        [-0.6268, -0.6276],
        [-0.6272, -0.6281],
        [-0.6277, -0.6285],
        [-0.6281, -0.6290],
        [-0.6285, -0